In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile,join
import re
import time
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import random
from sklearn.naive_bayes import MultinomialNB

训练集我们使用的是 IMDB 数据集。这个数据集包含 25000 条电影数据，其中 12500 条正向数据，12500 条负向数据。这些数据都是存储在一个文本文件中，首先我们需要做的就是去解析这个文件。正向数据包含在一个文件中，负向数据包含在另一个文件中。

In [2]:
positiveFiles = ["d:/input_data/aclImdb_v1/aclImdb/train/pos/" + f for f in listdir("d:/input_data/aclImdb_v1/aclImdb/train/pos/") if isfile(join("d:/input_data/aclImdb_v1/aclImdb/train/pos/",f))]
negativeFiles = ["d:/input_data/aclImdb_v1/aclImdb/train/neg/" + f for f in listdir("d:/input_data/aclImdb_v1/aclImdb/train/neg/") if isfile(join("d:/input_data/aclImdb_v1/aclImdb/train/neg/",f))]
numWords = []
for pf in positiveFiles:
    with open(pf,'r',encoding = "utf-8") as f:
        line = f.readline()
        counter = len(line.split())
        numWords.append(counter)
print("Positive files finished!")

for nf in negativeFiles:
    with open(nf,"r",encoding = "utf-8") as f:
        line = f.readline()
        counter = len(line.split())
        numWords.append(counter)
print("Negative files finished!")

numFiles = len(numWords)
print("The total number of line is ",numFiles)
print("the total number of words in the files is",sum(numWords))
print("the average number of words in the files is",sum(numWords)/len(numWords))


In [5]:
# 数据清洗
# 停用词
stopwords = {}.fromkeys([ line.rstrip() for line in open("d:/input_data/word2vec/kaggle-word2vec/stopwords.txt")])
eng_stopwords = set(stopwords)
def clean_text(text):
    text = BeautifulSoup(text,'html.parser').get_text()
    text = re.sub("[^a-zA-Z]"," ",text)
    words = text.lower().split()
    words = [ w for w in words if w not in eng_stopwords]
    return words


In [6]:
#英文分词模块
tokenizer = nltk.data.load("../nltk_data/tokenizers/punkt/english.pickle")
# 分句方法
def split_sentences(words):
    raw_sentences = tokenizer.tokenize(words.strip())
    sentences = [clean_text(s) for s in raw_sentences if s]
    return sentences

In [8]:
def makeWordsSet(words_file):
    words_set = set()
    with open(words_file,'r') as fp:
        for line in fp.readlines():
            cleanLine = clean_text(line)
            word = cleanLine.strip()
            if len(word)>0 and word not in words_set:
                words_set.add(word)
    return words_set

In [9]:
def TextProcessing(floder_path,test_size = 0.2):
    folder_list = os.listdir(folder_path)[1:3]
    data_list = []
    class_list = []
    
    # 类间循环
    for folder in folder_list:
        if folder == 'pos':
            flag = 1
        elif folder == 'neg':
            flag = 0
        new_folder_path = os.path.join(folder_path,folder)
        files = os.listdir(new_folder_path)
        
        # 类内循环
        for file in files:
            # 读取文件
            with open(os.path.join(new_folder_path,file),'r',encoding="utf-8") as fp:
                line = fp.readline()
                line = clean_text(line)
            data_list.append(line)
            class_list.append(flag)
            
    data_class_list = list(zip(data_list, class_list))
    np.random.shuffle(data_class_list)
    index = int(len(data_class_list)*test_size)+1
    train_list = data_class_list[index:]
    test_list = data_class_list[:index]
    train_data_list,train_class_list = zip(*train_list)
    test_data_list,test_class_list = zip(*test_list)
    
    # 统计词频放入all_words_dict
    all_words_dict = {}
    for word_list in train_data_list:
        for word in word_list:
            if word in all_words_dict:
                all_words_dict[word] += 1
            else:
                all_words_dict[word] = 1
    
    # key函数利用词频进行降序排序
    # 内建函数sorted参数需为list
    all_words_tuple_list = sorted(all_words_dict.items(), key=lambda f:f[1], reverse=True) # 内建函数sorted参数需为list
    all_words_list = list(zip(*all_words_tuple_list))[0]
    
    return all_words_dict,all_words_list,train_data_list,test_data_list,train_class_list,test_class_list

In [10]:
def words_dict(all_words_list,deleteN,N):
    # 特征词
    feature_words = []
    n = 1
    for t in range(deleteN,len(all_words_list),1):
        if n>N:
            break
        if 3<len(all_words_list[t])<20:
            feature_words.append(all_words_list[t])
            n+= 1
    return feature_words

In [33]:
def TextFeatures(train_data_list,test_data_list,feature_words,flag = 'nltk'):
    def text_features(text,feature_words):
        text_words = set(text)
        if flag == 'nltk':
            # nltk特征 dict
            features = {word:1 if word in text_words else 0 for word in feature_words}
        elif flag == 'sklearn':
            features = [1 if word in text_words else 0 for word in feature_words]
        else:
            features = []
        return features
    
    train_feature_list = [text_features(text,feature_words) for text in train_data_list]
    test_feature_list = [text_features(text,feature_words) for text in test_data_list]
    return train_feature_list,test_feature_list


In [12]:
def TextClassifier(train_feature_list,test_feature_list,train_class_list,test_class_list, flag='nltk'):
    if flag == 'nltk':
        # nltk 分类器
        train_flist = zip(train_feature_list,train_class_list)
        test_flist = zip(test_feature_list,test_class_list)
        classifier = nltk.classify.NaiveBayesClassifier.train(train_flist)
        test_accuracy = nltk.classify.accuracy(classifier,test_flist)
    elif flag == 'sklearn':
        # sklearn 分类器
        classifier = MultinomialNB().fit(train_feature_list,train_class_list)
        test_accuracy = classifier.score(test_feature_list,test_class_list)
    else:
        test_accuracy = []
    return test_accuracy,classifier

In [13]:
## 文本预处理
folder_path = "d:/input_data/aclImdb_v1/aclImdb/train/"
all_words_dict,all_words_list, train_data_list, test_data_list, train_class_list, test_class_list = TextProcessing(folder_path, test_size=0.2)

In [22]:
sentiment_to_num = {"pos":1,"neg":0}
flag = 'sklearn'
# test_accuracy_list = []
# deleteNS = range(0,100,20)
# i = 1
# for deleteN in deleteNS:
#     feature_words = words_dict(all_words_list,deleteN,N)
#     train_feature_list,test_feature_list = TextFeatures(train_data_list,test_data_list,feature_words, flag)
#     test_accuracy,classifier = TextClassifier(train_feature_list,test_feature_list,train_class_list, test_class_list, flag)
#     test_accuracy_list.append(test_accuracy)
#     print("第{}次训练完毕，准确率：{}".format(i,test_accuracy))
#     i += 1
N=2000
deleteN = 20
feature_words = words_dict(all_words_list,deleteN,N)
train_feature_list,test_feature_list = TextFeatures(train_data_list,test_data_list,feature_words, flag)
test_accuracy,classifier = TextClassifier(train_feature_list,test_feature_list,train_class_list, test_class_list, flag)
test_accuracy_list.append(test_accuracy)
print("准确率：{}".format(test_accuracy))

准确率：0.8370325934813038


In [51]:
# 为单个的评论创建文本特征
def testTextFeatures(comment_data,feature_words,flag = 'nltk'):
    data = np.array(comment_data).reshape(1,-1)
    def text_features(text,feature_words):
        text_words = set(text)
        if flag == 'nltk':
            # nltk特征 dict
            features = {word:1 if word in text_words else 0 for word in feature_words}
        elif flag == 'sklearn':
            features = [1 if word in text_words else 0 for word in feature_words]
        else:
            features = []
        return features
    
    data_feature = [text_features(text,feature_words) for text in data]
    return data_feature

In [52]:
# 测试单独的评论
def test_comment(comment_data,classifier,features,flag='sklean'):
    comment_data = clean_text(comment_data)
    data_feature = testTextFeatures(comment_data,feature_words, flag)
    pred = classifier.predict(data_feature)
    result = num_to_sentiment[int(pred)]
    return result

In [54]:
comment1 = 'the movie is very prefect,I love it very much,I will support it forever!'
comment2 = 'the movie is very terrible,I do not like it very much,I do not see it never!'
res = test_comment(comment2,classifier,feature_words,flag)
print(res)

neg
